### Run Asset Pool 1

In [1]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV, lang='english',check=False)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.26.4, server:0.27.6

#### Pool

In `Pool` , it will have a field named `assets` ,which contains a list of `assets`

In [2]:
mortgage1 = ["Mortgage"
            ,{"originBalance": 12000.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 40
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 10000.0
            ,"currentRate": 0.075
            ,"remainTerm": 30
            ,"status": "current"}]

mortgage2 = ["Mortgage"
            ,{"originBalance": 800.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 25
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 600.0
            ,"currentRate": 0.075
            ,"remainTerm": 15
            ,"status": "current"}]

myPool = {'assets':[ mortgage1, mortgage2  ],
         'cutoffDate':"2022-03-01"}

In [3]:
r = localAPI.runPool(myPool
                  ,poolAssump=("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                    ,None
                                    ,None)
                  ,read=True)

r['PoolConsol'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9542.79,345.80,61.80,0,7.62,0,7.62,0.075,None,None,345.80,0,0,7.62,0,7.62
2022-04-01,9186.98,347.68,59.58,0,8.13,0,8.13,0.075,None,None,693.48,0,0,15.75,0,15.75
2022-05-01,8829.85,349.56,57.36,0,7.57,0,7.57,0.075,None,None,1043.04,0,0,23.32,0,23.32
2022-06-01,8470.88,351.45,55.13,0,7.52,0,7.52,0.075,None,None,1394.49,0,0,30.84,0,30.84
2022-07-01,8110.54,353.35,52.89,0,6.99,0,6.99,0.075,None,None,1747.84,0,0,37.83,0,37.83
2022-08-01,7748.37,355.26,50.64,0,6.91,0,6.91,0.075,None,None,2103.10,0,0,44.74,0,44.74
2022-09-01,7384.60,357.17,48.38,0,6.60,0,6.60,0.075,None,None,2460.27,0,0,51.34,0,51.34
2022-10-01,7019.39,359.12,46.10,0,6.09,0,6.09,0.075,None,None,2819.39,0,0,57.43,0,57.43
2022-11-01,6652.36,361.05,43.82,0,5.98,0,5.98,0.075,None,None,3180.44,0,0,63.41,0,63.41


#### Cutoff day

`Cutoff day` means the day after which assets's cashflow belongs to the pool

In [ ]:
myPool = {'assets':[ mortgage1, mortgage2  ],
         'cutoffDate':"2023-03-01"}

r = localAPI.runPool(myPool
                  ,poolAssump=("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                    ,None
                                    ,None)
                  ,read=True)

r['PoolConsol'][0].head()

: 

#### Mixed Asset Pool

User can build multiple pools with different asset type

In [5]:
receivable1 = ["Invoice"
                  ,{"start":"2024-04-01","originBalance":2000
                    ,"originAdvance":1500,"dueDate":"2024-06-01"}
                  ,{"status":"Current"}]

myMixedPool = {"invoicePool": {'assets':[receivable1],}
              ,"mortgagePool": {'assets':[mortgage1]}
              ,'cutoffDate':"2022-03-01"}

Without any stress assumption, the return result will be a map, which keys are the pool name

In [6]:
r = localAPI.runPool(myMixedPool,read=True)

r['invoicePool'][0].head()

[('2024-04-01', 2000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 ('2024-06-01', 0, 0, 2000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]

In [7]:
r['mortgagePool'][0].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9082.00,307.91,58.68,0,0,0,0,0.075,None,None,307.91,0,0,0,0,0
2022-04-01,8772.17,309.83,56.76,0,0,0,0,0.075,None,None,617.74,0,0,0,0,0
2022-05-01,8460.40,311.77,54.82,0,0,0,0,0.075,None,None,929.51,0,0,0,0,0
2022-06-01,8146.68,313.72,52.87,0,0,0,0,0.075,None,None,1243.23,0,0,0,0,0
2022-07-01,7831.00,315.68,50.91,0,0,0,0,0.075,None,None,1558.91,0,0,0,0,0
2022-08-01,7513.35,317.65,48.94,0,0,0,0,0.075,None,None,1876.56,0,0,0,0,0
2022-09-01,7193.71,319.64,46.95,0,0,0,0,0.075,None,None,2196.20,0,0,0,0,0
2022-10-01,6872.08,321.63,44.96,0,0,0,0,0.075,None,None,2517.83,0,0,0,0,0
2022-11-01,6548.44,323.64,42.95,0,0,0,0,0.075,None,None,2841.47,0,0,0,0,0


Set assumptions to the pools
* passing a tuple with key ``ByPoolID`` to pool Assumption
* a map with key equals to the name of pool

In [8]:
mPoolAssump = ("Pool",
               ("Mortgage",{"CDR":0.01},None,None,None)
                ,None
                ,None)

iPoolAssump = ("Pool"
                ,("Receivable", {"CDR":0.01}, None, None)
                ,None
                ,None)


r = localAPI.runPool(myMixedPool
                  ,poolAssump = ("ByPoolId",
                                    {"invoicePool":iPoolAssump
                                    ,"mortgagePool":mPoolAssump}
                                    )
                  ,read=True)